In [1]:
%pylab inline
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms
from torchvision.datasets import MNIST

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = MNIST('./data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

test = MNIST('./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

# Create DataLoader
dataloader_args = dict(shuffle=True, batch_size=64,num_workers=1, pin_memory=True)
train_loader = dataloader.DataLoader(train, **dataloader_args)
test_loader = dataloader.DataLoader(test, **dataloader_args)

Processing...
Done!


In [3]:
train_data = train.train_data
train_data = train.transform(train_data.numpy())

print('[Train]')
print(' - Numpy Shape:', train.train_data.cpu().numpy().shape)
print(' - Tensor Shape:', train.train_data.size())
print(' - Transformed Shape:', train_data.size())
print(' - min:', torch.min(train_data))
print(' - max:', torch.max(train_data))
print(' - mean:', torch.mean(train_data))
print(' - std:', torch.std(train_data))
print(' - var:', torch.var(train_data))

[Train]
 - Numpy Shape: (60000, 28, 28)
 - Tensor Shape: torch.Size([60000, 28, 28])
 - Transformed Shape: torch.Size([28, 60000, 28])
 - min: 0.0
 - max: 1.0
 - mean: 0.13066047740240005
 - std: 0.3081078089011192
 - var: 0.0949304219058486


In [18]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.fc1 = nn.Linear(784, 548)
        self.bc1 = nn.BatchNorm1d(548)
        
        self.fc2 = nn.Linear(548, 252)
        self.bc2 = nn.BatchNorm1d(252)
        
        self.fc3 = nn.Linear(252, 10)
        
        
    def forward(self, x):
        x = x.view((-1, 784))
        h = self.fc1(x)
        h = self.bc1(h)
        h = F.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        
        h = self.fc2(h)
        h = self.bc2(h)
        h = F.relu(h)
        h = F.dropout(h, p=0.2, training=self.training)
        
        h = self.fc3(h)
        out = F.log_softmax(h)
        return out

model = Model()
#model.cuda() # CUDA!
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
model.train()


Model(
  (fc1): Linear(in_features=784, out_features=548)
  (bc1): BatchNorm1d(548, eps=1e-05, momentum=0.1, affine=True)
  (fc2): Linear(in_features=548, out_features=252)
  (bc2): BatchNorm1d(252, eps=1e-05, momentum=0.1, affine=True)
  (fc3): Linear(in_features=252, out_features=10)
)

In [21]:
losses = []
for epoch in range(15):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get Samples
        data, target = Variable(data), Variable(target)
        
        # Init
        optimizer.zero_grad()

        # Predict
        y_pred = model(data) 

        # Calculate loss
        loss = F.cross_entropy(y_pred, target)
        losses.append(loss.data[0])
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        
        # Display
        if batch_idx % 100 == 1:
            print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, 
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.data[0]), 
                end='')
            
    print()

/home/abc/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel/__main__.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


 Train Epoch: 0 [57664/60000 (96%)]	Loss: 0.168020
 Train Epoch: 1 [57664/60000 (96%)]	Loss: 0.065827
 Train Epoch: 2 [57664/60000 (96%)]	Loss: 0.063795
 Train Epoch: 3 [57664/60000 (96%)]	Loss: 0.125165
 Train Epoch: 4 [57664/60000 (96%)]	Loss: 0.065083
 Train Epoch: 5 [57664/60000 (96%)]	Loss: 0.276786
 Train Epoch: 6 [57664/60000 (96%)]	Loss: 0.021553
 Train Epoch: 7 [57664/60000 (96%)]	Loss: 0.042198
 Train Epoch: 8 [57664/60000 (96%)]	Loss: 0.012439
 Train Epoch: 9 [57664/60000 (96%)]	Loss: 0.044420
 Train Epoch: 10 [57664/60000 (96%)]	Loss: 0.169852
 Train Epoch: 11 [57664/60000 (96%)]	Loss: 0.079287
 Train Epoch: 12 [57664/60000 (96%)]	Loss: 0.036890
 Train Epoch: 13 [57664/60000 (96%)]	Loss: 0.024434
 Train Epoch: 14 [57664/60000 (96%)]	Loss: 0.030217
